# TMDB Box Office Predictions 
In this project, we will use supervised machine learning models to predict the box office revenue of hundreds of films

# API Pull 
To retrieve our dataset, we will conduct an API pull from the TMDB website 
https://www.themoviedb.org/

In [1]:
#import dependencies 
import pandas as pd
import numpy as np
from config import api_key
import json
from collections import Counter
from pprint import pprint
import requests
import os 
import csv 

In [64]:
data = pd.read_csv('train.csv')

In [65]:
data2= pd.read_csv('test.csv')

In [66]:
up_to_2019= pd.concat([data, data2])

Deciding features to know what we want to pull from TMDB API 

Features: film, budget, genres, original_language, popularity, production_companies, 
           release_date, runtime',revenue

Target: Revenue

In [6]:
#Kaggle Dataset Provides us with movie info up until 2019
#to keep project up to date, we will preform API pulls for 2020 and 2021 films 

In [7]:
#The tmdb only allows API pulls for 1 page at a time 
#page 1 url
#we will pull the 1st 10 pages of each year with descending revenue
response = requests.get('https://api.themoviedb.org/3/discover/movie?api_key=' +  api_key + '&primary_release_year=2020&sort_by=revenue.desc'+'&page=1')

In [8]:
highest_revenue = response.json() # store parsed json response

# uncomment the next line to get a peek at the highest_revenue json structure
# highest_revenue

highest_revenue_films = highest_revenue['results']

In [9]:
# define column names for our new dataframe
columns = ['film', 'budget', 'genres', 'original_language', 'popularity', 'production_companies', 'production_countries', 
           'release_date', 'runtime','revenue']

# create dataframe with film and revenue columns
df_2020 = pd.DataFrame(columns=columns)

In [10]:
# for each of the highest revenue films make an api call for that specific movie to return all of our column headers
for film in highest_revenue_films:
    # print(film['title'])
    film_revenue = requests.get('https://api.themoviedb.org/3/movie/'+ str(film['id']) +'?api_key='+ api_key+'&language=en-US')
    film_revenue = film_revenue.json()
    #print(locale.currency(film_revenue['revenue'], grouping=True ))
    df_2020.loc[len(df_2020)]=[film['title'],film_revenue['budget'],film_revenue['genres'],
                     film_revenue['original_language'],film_revenue['popularity'],
                     film_revenue['production_companies'],film_revenue['production_countries'],film_revenue['release_date'],film_revenue['runtime'],
                     film_revenue['revenue']] # store title and revenue in our dataframe    

In [11]:
#page 2
response2 = requests.get('https://api.themoviedb.org/3/discover/movie?api_key=' +  api_key + '&primary_release_year=2020&sort_by=revenue.desc'+'&page=2')

In [12]:
two= response2.json()
page_two = two['results']

In [13]:
df_2020_2 = pd.DataFrame(columns=columns)

In [14]:
# for each of the highest revenue films make an api call for that specific movie to return all of our column headers
for film in page_two:
    # print(film['title'])
    film_revenue = requests.get('https://api.themoviedb.org/3/movie/'+ str(film['id']) +'?api_key='+ api_key+'&language=en-US')
    film_revenue = film_revenue.json()
    #print(locale.currency(film_revenue['revenue'], grouping=True ))
    df_2020_2.loc[len(df_2020_2)]=[film['title'],film_revenue['budget'],film_revenue['genres'],
                     film_revenue['original_language'],film_revenue['popularity'],
                     film_revenue['production_companies'],film_revenue['production_countries'],film_revenue['release_date'],film_revenue['runtime'],
                     film_revenue['revenue']] # store title and revenue in our dataframe    

In [15]:
#page 3
response3 = requests.get('https://api.themoviedb.org/3/discover/movie?api_key=' +  api_key + '&primary_release_year=2020&sort_by=revenue.desc'+'&page=3')
three= response3.json()
page_three = three['results']

In [16]:
df_2020_3 = pd.DataFrame(columns=columns)

In [17]:
for film in page_three:
    # print(film['title'])
    film_revenue = requests.get('https://api.themoviedb.org/3/movie/'+ str(film['id']) +'?api_key='+ api_key+'&language=en-US')
    film_revenue = film_revenue.json()
    #print(locale.currency(film_revenue['revenue'], grouping=True ))
    df_2020_3.loc[len(df_2020_3)]=[film['title'],film_revenue['budget'],film_revenue['genres'],
                     film_revenue['original_language'],film_revenue['popularity'],
                     film_revenue['production_companies'],film_revenue['production_countries'],film_revenue['release_date'],film_revenue['runtime'],
                     film_revenue['revenue']] # store title and revenue in our dataframe 
    

In [18]:
#Page 4
response4 = requests.get('https://api.themoviedb.org/3/discover/movie?api_key=' +  api_key + '&primary_release_year=2020&sort_by=revenue.desc'+'&page=4')
four= response4.json()
page_four = four['results']
df_2020_4 = pd.DataFrame(columns=columns)

In [19]:
for film in page_four:
    # print(film['title'])
    film_revenue = requests.get('https://api.themoviedb.org/3/movie/'+ str(film['id']) +'?api_key='+ api_key+'&language=en-US')
    film_revenue = film_revenue.json()
    #print(locale.currency(film_revenue['revenue'], grouping=True ))
    df_2020_4.loc[len(df_2020_4)]=[film['title'],film_revenue['budget'],film_revenue['genres'],
                     film_revenue['original_language'],film_revenue['popularity'],
                     film_revenue['production_companies'],film_revenue['production_countries'],film_revenue['release_date'],film_revenue['runtime'],
                     film_revenue['revenue']] # store title and revenue in our dataframe 
    

In [20]:
#page5
response5 = requests.get('https://api.themoviedb.org/3/discover/movie?api_key=' +  api_key + '&primary_release_year=2020&sort_by=revenue.desc'+'&page=5')
five= response5.json()
page_five = five['results']
df_2020_5 = pd.DataFrame(columns=columns)

In [21]:
for film in page_five:
    # print(film['title'])
    film_revenue = requests.get('https://api.themoviedb.org/3/movie/'+ str(film['id']) +'?api_key='+ api_key+'&language=en-US')
    film_revenue = film_revenue.json()
    #print(locale.currency(film_revenue['revenue'], grouping=True ))
    df_2020_5.loc[len(df_2020_5)]=[film['title'],film_revenue['budget'],film_revenue['genres'],
                     film_revenue['original_language'],film_revenue['popularity'],
                     film_revenue['production_companies'],film_revenue['production_countries'],film_revenue['release_date'],film_revenue['runtime'],
                     film_revenue['revenue']] # store title and revenue in our dataframe 
    

In [22]:
#page6
response6 = requests.get('https://api.themoviedb.org/3/discover/movie?api_key=' +  api_key + '&primary_release_year=2020&sort_by=revenue.desc'+'&page=6')
six= response6.json()
page_six = six['results']
df_2020_6 = pd.DataFrame(columns=columns)

In [23]:
for film in page_six:
    # print(film['title'])
    film_revenue = requests.get('https://api.themoviedb.org/3/movie/'+ str(film['id']) +'?api_key='+ api_key+'&language=en-US')
    film_revenue = film_revenue.json()
    #print(locale.currency(film_revenue['revenue'], grouping=True ))
    df_2020_6.loc[len(df_2020_6)]=[film['title'],film_revenue['budget'],film_revenue['genres'],
                     film_revenue['original_language'],film_revenue['popularity'],
                     film_revenue['production_companies'],film_revenue['production_countries'],film_revenue['release_date'],film_revenue['runtime'],
                     film_revenue['revenue']] # store title and revenue in our dataframe 
    

In [24]:
#page7
response7 = requests.get('https://api.themoviedb.org/3/discover/movie?api_key=' +  api_key + '&primary_release_year=2020&sort_by=revenue.desc'+'&page=7')
seven= response7.json()
page_seven = seven['results']
df_2020_7 = pd.DataFrame(columns=columns)

In [25]:
for film in page_seven:
    # print(film['title'])
    film_revenue = requests.get('https://api.themoviedb.org/3/movie/'+ str(film['id']) +'?api_key='+ api_key+'&language=en-US')
    film_revenue = film_revenue.json()
    #print(locale.currency(film_revenue['revenue'], grouping=True ))
    df_2020_7.loc[len(df_2020_7)]=[film['title'],film_revenue['budget'],film_revenue['genres'],
                     film_revenue['original_language'],film_revenue['popularity'],
                     film_revenue['production_companies'],film_revenue['production_countries'],film_revenue['release_date'],film_revenue['runtime'],
                     film_revenue['revenue']] # store title and revenue in our dataframe 
    

In [26]:
#page8
response8 = requests.get('https://api.themoviedb.org/3/discover/movie?api_key=' +  api_key + '&primary_release_year=2020&sort_by=revenue.desc'+'&page=8')
eight= response8.json()
page_eight = eight['results']
df_2020_8 = pd.DataFrame(columns=columns)

In [27]:
for film in page_eight:
    # print(film['title'])
    film_revenue = requests.get('https://api.themoviedb.org/3/movie/'+ str(film['id']) +'?api_key='+ api_key+'&language=en-US')
    film_revenue = film_revenue.json()
    #print(locale.currency(film_revenue['revenue'], grouping=True ))
    df_2020_8.loc[len(df_2020_8)]=[film['title'],film_revenue['budget'],film_revenue['genres'],
                     film_revenue['original_language'],film_revenue['popularity'],
                     film_revenue['production_companies'],film_revenue['production_countries'],film_revenue['release_date'],film_revenue['runtime'],
                     film_revenue['revenue']] # store title and revenue in our dataframe 

In [28]:
#page9
response9 = requests.get('https://api.themoviedb.org/3/discover/movie?api_key=' +  api_key + '&primary_release_year=2020&sort_by=revenue.desc'+'&page=9')
nine= response9.json()
page_nine = nine['results']
df_2020_9 = pd.DataFrame(columns=columns)

In [29]:
for film in page_nine:
    # print(film['title'])
    film_revenue = requests.get('https://api.themoviedb.org/3/movie/'+ str(film['id']) +'?api_key='+ api_key+'&language=en-US')
    film_revenue = film_revenue.json()
    #print(locale.currency(film_revenue['revenue'], grouping=True ))
    df_2020_9.loc[len(df_2020_9)]=[film['title'],film_revenue['budget'],film_revenue['genres'],
                     film_revenue['original_language'],film_revenue['popularity'],
                     film_revenue['production_companies'],film_revenue['production_countries'],film_revenue['release_date'],film_revenue['runtime'],
                     film_revenue['revenue']] # store title and revenue in our dataframe 

In [30]:
#page10
response10 = requests.get('https://api.themoviedb.org/3/discover/movie?api_key=' +  api_key + '&primary_release_year=2020&sort_by=revenue.desc'+'&page=10')
ten= response10.json()
page_ten = ten['results']
df_2020_10 = pd.DataFrame(columns=columns)

In [31]:
for film in page_ten:
    # print(film['title'])
    film_revenue = requests.get('https://api.themoviedb.org/3/movie/'+ str(film['id']) +'?api_key='+ api_key+'&language=en-US')
    film_revenue = film_revenue.json()
    #print(locale.currency(film_revenue['revenue'], grouping=True ))
    df_2020_10.loc[len(df_2020_10)]=[film['title'],film_revenue['budget'],film_revenue['genres'],
                     film_revenue['original_language'],film_revenue['popularity'],
                     film_revenue['production_companies'],film_revenue['production_countries'],film_revenue['release_date'],film_revenue['runtime'],
                     film_revenue['revenue']] # store title and revenue in our dataframe 

In [32]:
#Combine all 10 2020 movie dfs 
Total_2020_Movies= pd.concat([df_2020, df_2020_2, df_2020_3, df_2020_4, df_2020_5, df_2020_6,
          df_2020_7, df_2020_8, df_2020_9, df_2020_10], ignore_index=True, axis=0)

Data for 2021

In [33]:
#page1
response1a = requests.get('https://api.themoviedb.org/3/discover/movie?api_key=' +  api_key + '&primary_release_year=2021&sort_by=revenue.desc'+'&page=1')
onea= response1a.json()
page_onea = onea['results']
df_2021 = pd.DataFrame(columns=columns)

In [34]:
for film in page_onea:
    # print(film['title'])
    film_revenue = requests.get('https://api.themoviedb.org/3/movie/'+ str(film['id']) +'?api_key='+ api_key+'&language=en-US')
    film_revenue = film_revenue.json()
    #print(locale.currency(film_revenue['revenue'], grouping=True ))
    df_2021.loc[len(df_2021)]=[film['title'],film_revenue['budget'],film_revenue['genres'],
                     film_revenue['original_language'],film_revenue['popularity'],
                     film_revenue['production_companies'],film_revenue['production_countries'],film_revenue['release_date'],film_revenue['runtime'],
                     film_revenue['revenue']] # store title and revenue in our dataframe 

In [35]:
#page2
response2a = requests.get('https://api.themoviedb.org/3/discover/movie?api_key=' +  api_key + '&primary_release_year=2021&sort_by=revenue.desc'+'&page=2')
twoa= response2a.json()
page_twoa = twoa['results']
df_2021_2 = pd.DataFrame(columns=columns)

In [36]:
for film in page_twoa:
    # print(film['title'])
    film_revenue = requests.get('https://api.themoviedb.org/3/movie/'+ str(film['id']) +'?api_key='+ api_key+'&language=en-US')
    film_revenue = film_revenue.json()
    #print(locale.currency(film_revenue['revenue'], grouping=True ))
    df_2021_2.loc[len(df_2021_2)]=[film['title'],film_revenue['budget'],film_revenue['genres'],
                     film_revenue['original_language'],film_revenue['popularity'],
                     film_revenue['production_companies'],film_revenue['production_countries'],film_revenue['release_date'],film_revenue['runtime'],
                     film_revenue['revenue']] # store title and revenue in our dataframe 

In [37]:
#page3
response3a = requests.get('https://api.themoviedb.org/3/discover/movie?api_key=' +  api_key + '&primary_release_year=2021&sort_by=revenue.desc'+'&page=2')
threea= response3a.json()
page_threea = threea['results']
df_2021_3 = pd.DataFrame(columns=columns)

In [38]:
for film in page_threea:
    # print(film['title'])
    film_revenue = requests.get('https://api.themoviedb.org/3/movie/'+ str(film['id']) +'?api_key='+ api_key+'&language=en-US')
    film_revenue = film_revenue.json()
    #print(locale.currency(film_revenue['revenue'], grouping=True ))
    df_2021_3.loc[len(df_2021_3)]=[film['title'],film_revenue['budget'],film_revenue['genres'],
                     film_revenue['original_language'],film_revenue['popularity'],
                     film_revenue['production_companies'],film_revenue['production_countries'],film_revenue['release_date'],film_revenue['runtime'],
                     film_revenue['revenue']] # store title and revenue in our dataframe 

In [39]:
#page4
response4a = requests.get('https://api.themoviedb.org/3/discover/movie?api_key=' +  api_key + '&primary_release_year=2021&sort_by=revenue.desc'+'&page=2')
foura= response4a.json()
page_foura = foura['results']
df_2021_4 = pd.DataFrame(columns=columns)

In [40]:
for film in page_foura:
    # print(film['title'])
    film_revenue = requests.get('https://api.themoviedb.org/3/movie/'+ str(film['id']) +'?api_key='+ api_key+'&language=en-US')
    film_revenue = film_revenue.json()
    #print(locale.currency(film_revenue['revenue'], grouping=True ))
    df_2021_4.loc[len(df_2021_4)]=[film['title'],film_revenue['budget'],film_revenue['genres'],
                     film_revenue['original_language'],film_revenue['popularity'],
                     film_revenue['production_companies'],film_revenue['production_countries'],film_revenue['release_date'],film_revenue['runtime'],
                     film_revenue['revenue']] # store title and revenue in our dataframe 

In [41]:
#page5
response5a = requests.get('https://api.themoviedb.org/3/discover/movie?api_key=' +  api_key + '&primary_release_year=2021&sort_by=revenue.desc'+'&page=2')
fivea= response5a.json()
page_fivea = fivea['results']
df_2021_5 = pd.DataFrame(columns=columns)

In [42]:
for film in page_twoa:
    # print(film['title'])
    film_revenue = requests.get('https://api.themoviedb.org/3/movie/'+ str(film['id']) +'?api_key='+ api_key+'&language=en-US')
    film_revenue = film_revenue.json()
    #print(locale.currency(film_revenue['revenue'], grouping=True ))
    df_2021_5.loc[len(df_2021_5)]=[film['title'],film_revenue['budget'],film_revenue['genres'],
                     film_revenue['original_language'],film_revenue['popularity'],
                     film_revenue['production_companies'],film_revenue['production_countries'],film_revenue['release_date'],film_revenue['runtime'],
                     film_revenue['revenue']] # store title and revenue in our dataframe 

In [43]:
#page6
response6a = requests.get('https://api.themoviedb.org/3/discover/movie?api_key=' +  api_key + '&primary_release_year=2021&sort_by=revenue.desc'+'&page=2')
sixa= response6a.json()
page_sixa = sixa['results']
df_2021_6 = pd.DataFrame(columns=columns)

In [44]:
for film in page_sixa:
    # print(film['title'])
    film_revenue = requests.get('https://api.themoviedb.org/3/movie/'+ str(film['id']) +'?api_key='+ api_key+'&language=en-US')
    film_revenue = film_revenue.json()
    #print(locale.currency(film_revenue['revenue'], grouping=True ))
    df_2021_6.loc[len(df_2021_6)]=[film['title'],film_revenue['budget'],film_revenue['genres'],
                     film_revenue['original_language'],film_revenue['popularity'],
                     film_revenue['production_companies'],film_revenue['production_countries'],film_revenue['release_date'],film_revenue['runtime'],
                     film_revenue['revenue']] # store title and revenue in our dataframe 

In [45]:
#page7
response7a = requests.get('https://api.themoviedb.org/3/discover/movie?api_key=' +  api_key + '&primary_release_year=2021&sort_by=revenue.desc'+'&page=2')
sevena= response7a.json()
page_sevena = sevena['results']
df_2021_7 = pd.DataFrame(columns=columns)

In [46]:
for film in page_sevena:
    # print(film['title'])
    film_revenue = requests.get('https://api.themoviedb.org/3/movie/'+ str(film['id']) +'?api_key='+ api_key+'&language=en-US')
    film_revenue = film_revenue.json()
    #print(locale.currency(film_revenue['revenue'], grouping=True ))
    df_2021_7.loc[len(df_2021_7)]=[film['title'],film_revenue['budget'],film_revenue['genres'],
                     film_revenue['original_language'],film_revenue['popularity'],
                     film_revenue['production_companies'],film_revenue['production_countries'],film_revenue['release_date'],film_revenue['runtime'],
                     film_revenue['revenue']] # store title and revenue in our dataframe 

In [47]:
#page8
response8a = requests.get('https://api.themoviedb.org/3/discover/movie?api_key=' +  api_key + '&primary_release_year=2021&sort_by=revenue.desc'+'&page=2')
eighta= response8a.json()
page_eighta = eighta['results']
df_2021_8 = pd.DataFrame(columns=columns)

In [48]:
for film in page_eighta:
    # print(film['title'])
    film_revenue = requests.get('https://api.themoviedb.org/3/movie/'+ str(film['id']) +'?api_key='+ api_key+'&language=en-US')
    film_revenue = film_revenue.json()
    #print(locale.currency(film_revenue['revenue'], grouping=True ))
    df_2021_8.loc[len(df_2021_8)]=[film['title'],film_revenue['budget'],film_revenue['genres'],
                     film_revenue['original_language'],film_revenue['popularity'],
                     film_revenue['production_companies'],film_revenue['production_countries'],film_revenue['release_date'],film_revenue['runtime'],
                     film_revenue['revenue']] # store title and revenue in our dataframe 

In [49]:
#page9
response9a = requests.get('https://api.themoviedb.org/3/discover/movie?api_key=' +  api_key + '&primary_release_year=2021&sort_by=revenue.desc'+'&page=2')
ninea= response9a.json()
page_ninea = ninea['results']
df_2021_9 = pd.DataFrame(columns=columns)

In [50]:
for film in page_ninea:
    # print(film['title'])
    film_revenue = requests.get('https://api.themoviedb.org/3/movie/'+ str(film['id']) +'?api_key='+ api_key+'&language=en-US')
    film_revenue = film_revenue.json()
    #print(locale.currency(film_revenue['revenue'], grouping=True ))
    df_2021_9.loc[len(df_2021_9)]=[film['title'],film_revenue['budget'],film_revenue['genres'],
                     film_revenue['original_language'],film_revenue['popularity'],
                     film_revenue['production_companies'],film_revenue['production_countries'],film_revenue['release_date'],film_revenue['runtime'],
                     film_revenue['revenue']] # store title and revenue in our dataframe 

In [51]:
#page10
response10a = requests.get('https://api.themoviedb.org/3/discover/movie?api_key=' +  api_key + '&primary_release_year=2021&sort_by=revenue.desc'+'&page=2')
tena= response10a.json()
page_tena = tena['results']
df_2021_10 = pd.DataFrame(columns=columns)

In [52]:
for film in page_tena:
    # print(film['title'])
    film_revenue = requests.get('https://api.themoviedb.org/3/movie/'+ str(film['id']) +'?api_key='+ api_key+'&language=en-US')
    film_revenue = film_revenue.json()
    #print(locale.currency(film_revenue['revenue'], grouping=True ))
    df_2021_10.loc[len(df_2021_10)]=[film['title'],film_revenue['budget'],film_revenue['genres'],
                     film_revenue['original_language'],film_revenue['popularity'],
                     film_revenue['production_companies'],film_revenue['production_countries'],film_revenue['release_date'],film_revenue['runtime'],
                     film_revenue['revenue']] # store title and revenue in our dataframe 

In [53]:
#Combine all 10 2021 movie dfs 
Total_2021_Movies = pd.concat([df_2021, df_2021_2, df_2021_3, df_2021_4, df_2021_5, df_2021_6,
          df_2021_7, df_2021_8, df_2021_9, df_2021_10], ignore_index=True, axis=0)

In [56]:
Total_Movies=  pd.concat([Total_2020_Movies, Total_2021_Movies])

# Editing the Release Date column so it matches the Kaggle Dataset

In [57]:
Total_Movies['release_date'] = pd.to_datetime(Total_Movies.release_date, format='%Y-%m-%d')

In [58]:
Total_Movies['release_date'] = Total_Movies["release_date"].dt.strftime("%m/%d/%y")

# Importing Kaggle Dataset 
https://www.kaggle.com/c/tmdb-box-office-prediction/data?select=train.csv
- Import data
- Concat with API data 
- Clean and reformat columns

In [60]:
#make all columns visible
pd.set_option('display.max_columns', None)

In [67]:
#drop columns we know will not aid our models 
up_to_2019 = up_to_2019.drop(columns=['belongs_to_collection', 'id', 'homepage','imdb_id','overview',
                                      'spoken_languages','status','tagline','original_title','Keywords','cast',
                                     'crew','poster_path'])

In [68]:
#reformat columns so they match API dataframes
up_to_2019 = up_to_2019[['title','budget','genres','production_companies','production_countries','original_language','popularity',
                         'release_date','runtime','revenue']]


In [69]:
#make sure headers match
up_to_2019.rename(columns={'title': 'film'}, inplace=True)

In [116]:
#concat all frames to get 1 large dataset
all_movies= pd.concat([up_to_2019, Total_Movies], ignore_index=True, axis=0)

In [117]:
all_movies["genres"] = all_movies.genres.astype(str)

In [118]:
def proc_json(string, key):
    try:
        data = eval(string)
        return ",".join([d[key] for d in data])
    except:
        return ''
    
all_movies.genres = all_movies.genres.apply(lambda x: proc_json(x, 'name'))

In [119]:
#turning genre category into numeric 
genres = []
for idx, val in all_movies.genres.iteritems():
    gen_list = val.split(',')
    for gen in gen_list:
        if gen == '':
            continue

        if gen not in genres:
            genres.append(gen)

In [121]:
export_movie_df= all_movies.copy()

# Data to keep for Tableau

In [123]:
export_movie_df["production_countries"] = all_movies.production_countries.astype(str)
export_movie_df["production_companies"] = all_movies.production_companies.astype(str)

In [124]:
def proc_json(string, key):
    try:
        data = eval(string)
        return ",".join([d[key] for d in data])
    except:
        return ''
    
export_movie_df.production_countries = export_movie_df.production_countries.apply(lambda x: proc_json(x, 'name'))
export_movie_df.production_companies = export_movie_df.production_companies.apply(lambda x: proc_json(x, 'name'))

In [125]:
export_movie_df

,film,budget,genres,production_companies,production_countries,original_language,popularity,release_date,runtime,revenue
0,Hot Tub Time Machine 2,14000000,Comedy,"Paramount Pictures,United Artists,Metro-Goldwy...",United States of America,en,6.575393,2/20/15,93.0,12314651.0
1,The Princess Diaries 2: Royal Engagement,40000000,"Comedy,Drama,Family,Romance",Walt Disney Pictures,United States of America,en,8.248895,8/6/04,113.0,95149435.0
2,Whiplash,3300000,Drama,"Bold Films,Blumhouse Productions,Right of Way ...",United States of America,en,64.299990,10/10/14,105.0,13092000.0
3,Kahaani,1200000,"Thriller,Drama",,India,hi,3.174936,3/9/12,122.0,16000000.0
4,Marine Boy,0,"Action,Thriller",,South Korea,ko,1.148070,2/5/09,118.0,3923970.0
...,...,...,...,...,...,...,...,...,...,...
7793,Halloween Kills,20000000,"Horror,Thriller","Universal Pictures,Blumhouse Productions,Home ...","United Kingdom,United States of America",en,336.130000,10/14/21,105,127000000
7794,The King's Man,100000000,"Action,Adventure,Thriller,War","Marv Films,Cloudy Productions,20th Century Stu...","United Kingdom,United States of America",en,3507.400000,12/22/21,131,124005195
7795,The Addams Family 2,0,"Animation,Adventure,Comedy,Family","Metro-Goldwyn-Mayer,Bron Studios,BermanBraun,C...","Canada,United States of America",en,376.331000,10/01/21,93,119815153
7796,Wrath of Man,40000000,"Action,Crime,Thriller","Miramax,Metro-Goldwyn-Mayer,Toff Guy Films,Lio...","United Kingdom,United States of America",en,379.364000,04/22/21,119,103966489


In [126]:
export_movie_df['genre_1'] = export_movie_df['genres'].str.split(',').str[0]
export_movie_df['production_company'] = export_movie_df['production_companies'].str.split(',').str[0]
export_movie_df['production_country'] = export_movie_df['production_countries'].str.split(',').str[0]

In [129]:
export_movie_df = export_movie_df[['film', 'budget', 'genres','genre_1','production_companies',
                       'production_company','production_countries','production_country',
                       'original_language', 'popularity', 'release_date',
                       'runtime', 'revenue']]

In [130]:
export_movie_df= export_movie_df.drop(columns=['genres','production_companies','production_countries'])

In [131]:
export_movie_df

,film,budget,genre_1,production_company,production_country,original_language,popularity,release_date,runtime,revenue
0,Hot Tub Time Machine 2,14000000,Comedy,Paramount Pictures,United States of America,en,6.575393,2/20/15,93.0,12314651.0
1,The Princess Diaries 2: Royal Engagement,40000000,Comedy,Walt Disney Pictures,United States of America,en,8.248895,8/6/04,113.0,95149435.0
2,Whiplash,3300000,Drama,Bold Films,United States of America,en,64.299990,10/10/14,105.0,13092000.0
3,Kahaani,1200000,Thriller,,India,hi,3.174936,3/9/12,122.0,16000000.0
4,Marine Boy,0,Action,,South Korea,ko,1.148070,2/5/09,118.0,3923970.0
...,...,...,...,...,...,...,...,...,...,...
7793,Halloween Kills,20000000,Horror,Universal Pictures,United Kingdom,en,336.130000,10/14/21,105,127000000
7794,The King's Man,100000000,Action,Marv Films,United Kingdom,en,3507.400000,12/22/21,131,124005195
7795,The Addams Family 2,0,Animation,Metro-Goldwyn-Mayer,Canada,en,376.331000,10/01/21,93,119815153
7796,Wrath of Man,40000000,Action,Miramax,United Kingdom,en,379.364000,04/22/21,119,103966489


In [132]:
export_movie_df.to_csv('export_movie_df.csv')

In [178]:
all_movies= export_movie_df.copy()

# Addressing Missing Values

In [179]:
all_movies.isnull().sum()

film                     3
budget                   0
genre_1                  0
production_company       0
production_country       0
original_language        0
popularity               0
release_date             1
runtime                  6
revenue               4398
dtype: int64

In [180]:
#Keep in mind that all the categorical variables are nominal so they will need to be onehotencoded NOT Intenger Encoding

In [181]:
#Replacing With Mean
#This is the most common method of imputing missing values of numeric columns. 
#If there are outliers then the mean will not be appropriate. 
#In such cases, outliers need to be treated first.
#filling empty revenue columns with average revenue 
all_movies['revenue']= all_movies['revenue'].fillna(all_movies['revenue'].median())
all_movies['budget']= all_movies['budget'].fillna(all_movies['budget'].median())
#Replacing With Mode
#Mode is the most frequently occurring value. It is used in the case of categorical features.
all_movies['production_company'] = all_movies['production_company'].fillna(all_movies['production_company'].mode()[0])

In [182]:
#adding index column so we can remove the film name for scaling before splitting into test and train so the indexes remain true
all_movies['index'] = all_movies.index

In [183]:
all_movies = all_movies[['index', 'film', 'budget', 'genre_1', 'production_company','production_country','original_language', 'popularity', 'release_date',
                       'runtime', 'revenue']]

In [184]:
budget_median = all_movies.loc[all_movies['budget'] > 0, 'budget'].median()
all_movies["budget_processed"] = all_movies["budget"].mask(all_movies["budget"] == 0, budget_median)

In [185]:
all_movies = all_movies[['index', 'film', 'budget', 'budget_processed','genre_1', 'production_company',
                         'production_country','original_language', 
                         'popularity', 'release_date',
                       'runtime', 'revenue']]

In [186]:
all_movies= all_movies.drop(columns=['budget'])

In [187]:
#fill in missing value with random date
all_movies['release_date'] = all_movies['release_date'].fillna('6/15/10')

In [188]:
import re
def yearfix(x): # run year fix, then date fix
    """Regular expression to pull out the two digit year from release_date and return it as an int"""
    r = re.match(r"(\d+/\d+/)(\d+)",x)[2]
    return int(r)
def datefix(x):
    """The dates only provide two digits for year.  This is meant to fix this issue.
    The youngest movie is from 2021, so we'll say any year digits less than 21 are from this century.
    Otherwise, we'll say they are from the 1900s."""
    if x<21:
        x = x+2000
        return x
    if x >=21:
        x = x+1900
        return x

# Machine Learning Data Preprocessing

In [189]:
#import sklearn dependencies 
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans

In [190]:
all_movies.shape

(7798, 11)

In [191]:
from numpy import mean
import time
from datetime import datetime
import calendar
import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns
import re
from scipy.stats import pearsonr
import math
from statistics import median
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor

In [192]:
all_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7798 entries, 0 to 7797
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   index               7798 non-null   int64  
 1   film                7795 non-null   object 
 2   budget_processed    7798 non-null   float64
 3   genre_1             7798 non-null   object 
 4   production_company  7798 non-null   object 
 5   production_country  7798 non-null   object 
 6   original_language   7798 non-null   object 
 7   popularity          7798 non-null   float64
 8   release_date        7798 non-null   object 
 9   runtime             7792 non-null   object 
 10  revenue             7798 non-null   float64
dtypes: float64(3), int64(1), object(7)
memory usage: 670.3+ KB


In [193]:
all_movies= all_movies.dropna()

In [194]:
all_movies.isnull().sum()

index                 0
film                  0
budget_processed      0
genre_1               0
production_company    0
production_country    0
original_language     0
popularity            0
release_date          0
runtime               0
revenue               0
dtype: int64

In [195]:
#converting objects that are currently numeric to float 
#converting release date to datetime
all_movies["index"] = all_movies.index.astype(float)
all_movies["budget_processed"] = all_movies.budget_processed.astype(float)
all_movies["runtime"] = all_movies.runtime.astype(float)
all_movies["revenue"] = all_movies.revenue.astype(float)
all_movies["release_date"] = pd.to_datetime(all_movies["release_date"])

In [196]:
all_movies['release_date'] = all_movies[('release_date')].values.astype("float64")

In [197]:
all_movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7789 entries, 0 to 7797
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   index               7789 non-null   float64
 1   film                7789 non-null   object 
 2   budget_processed    7789 non-null   float64
 3   genre_1             7789 non-null   object 
 4   production_company  7789 non-null   object 
 5   production_country  7789 non-null   object 
 6   original_language   7789 non-null   object 
 7   popularity          7789 non-null   float64
 8   release_date        7789 non-null   float64
 9   runtime             7789 non-null   float64
 10  revenue             7789 non-null   float64
dtypes: float64(6), object(5)
memory usage: 730.2+ KB


# Converting Categorical Data to numeric with OneHotEncoding 
We need to convert:
- genre
- production company
- production country
- original language 

In [198]:
#rename genre_1 to genres
all_movies.rename(columns={'genre_1': 'genres'}, inplace=True)

In [199]:
#creating numerical data for the genre column 
genre_column_names = []
for gen in genres:
    col_name = 'genre_' + gen.replace(' ', '_')
    all_movies[col_name] = all_movies.genres.str.contains(gen).astype('uint8')
    genre_column_names.append(col_name)

In [200]:
all_movies = pd.get_dummies(all_movies, columns = ["original_language"])

In [201]:
all_movies

,index,film,budget_processed,genres,production_company,production_country,popularity,release_date,runtime,revenue,genre_Comedy,genre_Drama,genre_Family,genre_Romance,genre_Thriller,genre_Action,genre_Animation,genre_Adventure,genre_Horror,genre_Documentary,genre_Music,genre_Crime,genre_Science_Fiction,genre_Mystery,genre_Foreign,genre_Fantasy,genre_War,genre_Western,genre_History,genre_TV_Movie,original_language_af,original_language_ar,original_language_bm,original_language_bn,original_language_ca,original_language_cn,original_language_cs,original_language_da,original_language_de,original_language_el,original_language_en,original_language_es,original_language_fa,original_language_fi,original_language_fr,original_language_he,original_language_hi,original_language_hu,original_language_id,original_language_is,original_language_it,original_language_ja,original_language_ka,original_language_kn,original_language_ko,original_language_lt,original_language_ml,original_language_mr,original_language_nb,original_language_nl,original_language_no,original_language_pl,original_language_pt,original_language_ro,original_language_ru,original_language_sr,original_language_sv,original_language_ta,original_language_te,original_language_th,original_language_tl,original_language_tr,original_language_uk,original_language_ur,original_language_vi,original_language_xx,original_language_zh
0,0.0,Hot Tub Time Machine 2,14000000.0,Comedy,Paramount Pictures,United States of America,6.575393,1.424390e+18,93.0,12314651.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1.0,The Princess Diaries 2: Royal Engagement,40000000.0,Comedy,Walt Disney Pictures,United States of America,8.248895,1.091750e+18,113.0,95149435.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2.0,Whiplash,3300000.0,Drama,Bold Films,United States of America,64.299990,1.412899e+18,105.0,13092000.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3.0,Kahaani,1200000.0,Thriller,,India,3.174936,1.331251e+18,122.0,16000000.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,4.0,Marine Boy,18000000.0,Action,,South Korea,1.148070,1.233792e+18,118.0,3923970.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7793,7793.0,Halloween Kills,20000000.0,Horror,Universal Pictures,United Kingdom,336.130000,1.634170e+18,105.0,127000000.0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7794,7794.0,The King's Man,100000000.0,Action,Marv Films,United Kingdom,3507.400000,1.640131e+18,131.0,124005195.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7795,7795.0,The Addams Family 2,18000000.0,Animation,Metro-Goldwyn-Mayer,Canada,376.331000,1.633046e+18,93.0,119815153.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7796,7796.0,Wrath of Man,40000000.0,Action,Miramax,United Kingdom,379.364000,1.619050e+18,119.0,103966489.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [205]:
pls_work= all_movies.copy()

# Train and Test Split

Next we will need to perform a test train split. 
After data is split drop the revenue column from the test data because that will be our target variable in our models

In [206]:
pls_work= pls_work.drop(columns=['film'])
pls_work= pls_work.drop(columns=['genres'])
pls_work= pls_work.drop(columns=['production_company'])
pls_work= pls_work.drop(columns=['production_country'])

In [207]:
train=pls_work.sample(frac=0.75,random_state=42) #random state is a seed value
test=pls_work.drop(train.index)

In [128]:
train=all_movies.sample(frac=0.75,random_state=42) #random state is a seed value
test=all_movies.drop(train.index)

In [208]:
y_train = train["revenue"]
X_train = train.drop(columns = ["revenue"])
X_train.head()

,index,budget_processed,popularity,release_date,runtime,genre_Comedy,genre_Drama,genre_Family,genre_Romance,genre_Thriller,genre_Action,genre_Animation,genre_Adventure,genre_Horror,genre_Documentary,genre_Music,genre_Crime,genre_Science_Fiction,genre_Mystery,genre_Foreign,genre_Fantasy,genre_War,genre_Western,genre_History,genre_TV_Movie,original_language_af,original_language_ar,original_language_bm,original_language_bn,original_language_ca,original_language_cn,original_language_cs,original_language_da,original_language_de,original_language_el,original_language_en,original_language_es,original_language_fa,original_language_fi,original_language_fr,original_language_he,original_language_hi,original_language_hu,original_language_id,original_language_is,original_language_it,original_language_ja,original_language_ka,original_language_kn,original_language_ko,original_language_lt,original_language_ml,original_language_mr,original_language_nb,original_language_nl,original_language_no,original_language_pl,original_language_pt,original_language_ro,original_language_ru,original_language_sr,original_language_sv,original_language_ta,original_language_te,original_language_th,original_language_tl,original_language_tr,original_language_uk,original_language_ur,original_language_vi,original_language_xx,original_language_zh
5839,5839.0,1.0,2.240259,1.449187e+18,114.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5944,5944.0,18000000.0,10.087404,1.042848e+18,108.0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1323,1323.0,18000000.0,1.935006,7.833024e+17,102.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5113,5113.0,2135161.0,6.409094,2.982787e+18,130.0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
101,101.0,60000000.0,11.936460,1.008029e+18,135.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [209]:
#drop Revenue column from TEST df because that will be our target variable
y_test = test["revenue"]
X_test = test.drop(columns = ["revenue"])
X_test.head()

,index,budget_processed,popularity,release_date,runtime,genre_Comedy,genre_Drama,genre_Family,genre_Romance,genre_Thriller,genre_Action,genre_Animation,genre_Adventure,genre_Horror,genre_Documentary,genre_Music,genre_Crime,genre_Science_Fiction,genre_Mystery,genre_Foreign,genre_Fantasy,genre_War,genre_Western,genre_History,genre_TV_Movie,original_language_af,original_language_ar,original_language_bm,original_language_bn,original_language_ca,original_language_cn,original_language_cs,original_language_da,original_language_de,original_language_el,original_language_en,original_language_es,original_language_fa,original_language_fi,original_language_fr,original_language_he,original_language_hi,original_language_hu,original_language_id,original_language_is,original_language_it,original_language_ja,original_language_ka,original_language_kn,original_language_ko,original_language_lt,original_language_ml,original_language_mr,original_language_nb,original_language_nl,original_language_no,original_language_pl,original_language_pt,original_language_ro,original_language_ru,original_language_sr,original_language_sv,original_language_ta,original_language_te,original_language_th,original_language_tl,original_language_tr,original_language_uk,original_language_ur,original_language_vi,original_language_xx,original_language_zh
2,2.0,3300000.0,64.299990,1.412899e+18,105.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3.0,1200000.0,3.174936,1.331251e+18,122.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,4.0,18000000.0,1.148070,1.233792e+18,118.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,5.0,8000000.0,0.743274,5.552064e+17,83.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,9.0,6000000.0,4.672036,1.050451e+18,91.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [210]:
#take a look at the shape of our data
print ('The train data has {} rows and {} columns'.format(train.shape[0],train.shape[1]))
print ('---------------------------------------------')
print ('The test data has {} rows and {} columns'.format(test.shape[0],test.shape[1]))

The train data has 5842 rows and 73 columns
---------------------------------------------
The test data has 1947 rows and 73 columns


In [211]:
scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [212]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [213]:
# add missing dummy variables to testing set (in training only)
for column in X_train.columns:
    if column not in X_test.columns:
        X_test[column] = 0

In [215]:
reg = RandomForestClassifier().fit(X_train, y_train)
reg.score(X_test, y_test)

0.5963020030816641

In [131]:
#removing film from dataframes for scaling purposes
train = train.drop(columns=['film'])
test= test.drop(columns=['film'])

In [132]:
#dropping genres column since the data is now numerical
train = train.drop(columns=['genres'])
test = test.drop(columns=['genres'])

In [133]:
y_train = train["revenue"]
X_train = train.drop(columns = ["revenue"])
X_train.head()

,index,budget_processed,popularity,release_date,runtime,genre_Comedy,genre_Drama,genre_Family,genre_Romance,genre_Thriller,genre_Action,genre_Animation,genre_Adventure,genre_Horror,genre_Documentary,genre_Music,genre_Crime,genre_Science_Fiction,genre_Mystery,genre_Foreign,genre_Fantasy,genre_War,genre_Western,genre_History,genre_TV_Movie,original_language_af,original_language_ar,original_language_bm,original_language_bn,original_language_ca,original_language_cn,original_language_cs,original_language_da,original_language_de,original_language_el,original_language_en,original_language_es,original_language_fa,original_language_fi,original_language_fr,original_language_he,original_language_hi,original_language_hu,original_language_id,original_language_is,original_language_it,original_language_ja,original_language_ka,original_language_kn,original_language_ko,original_language_lt,original_language_ml,original_language_mr,original_language_nb,original_language_nl,original_language_no,original_language_pl,original_language_pt,original_language_ro,original_language_ru,original_language_sr,original_language_sv,original_language_ta,original_language_te,original_language_th,original_language_tl,original_language_tr,original_language_uk,original_language_ur,original_language_vi,original_language_xx,original_language_zh
2926,2926.0,18000000.0,5.325766,1.188346e+18,90.0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5502,5502.0,28000000.0,2.304986,7.975584e+17,74.0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2809,2809.0,35000000.0,8.910462,7.851168e+17,89.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1025,1025.0,18000000.0,5.819403,1.383696e+18,91.0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
101,101.0,60000000.0,11.936460,1.008029e+18,135.0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [134]:
#drop Revenue column from TEST df because that will be our target variable
y_test = test["revenue"]
X_test = test.drop(columns = ["revenue"])
X_test.head()

,index,budget_processed,popularity,release_date,runtime,genre_Comedy,genre_Drama,genre_Family,genre_Romance,genre_Thriller,genre_Action,genre_Animation,genre_Adventure,genre_Horror,genre_Documentary,genre_Music,genre_Crime,genre_Science_Fiction,genre_Mystery,genre_Foreign,genre_Fantasy,genre_War,genre_Western,genre_History,genre_TV_Movie,original_language_af,original_language_ar,original_language_bm,original_language_bn,original_language_ca,original_language_cn,original_language_cs,original_language_da,original_language_de,original_language_el,original_language_en,original_language_es,original_language_fa,original_language_fi,original_language_fr,original_language_he,original_language_hi,original_language_hu,original_language_id,original_language_is,original_language_it,original_language_ja,original_language_ka,original_language_kn,original_language_ko,original_language_lt,original_language_ml,original_language_mr,original_language_nb,original_language_nl,original_language_no,original_language_pl,original_language_pt,original_language_ro,original_language_ru,original_language_sr,original_language_sv,original_language_ta,original_language_te,original_language_th,original_language_tl,original_language_tr,original_language_uk,original_language_ur,original_language_vi,original_language_xx,original_language_zh
2,2.0,3300000.0,64.299990,1.412899e+18,105.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3.0,1200000.0,3.174936,1.331251e+18,122.0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,4.0,18000000.0,1.148070,1.233792e+18,118.0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,5.0,8000000.0,0.743274,5.552064e+17,83.0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,9.0,6000000.0,4.672036,1.050451e+18,91.0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [135]:
#take a look at the shape of our data
print ('The train data has {} rows and {} columns'.format(train.shape[0],train.shape[1]))
print ('---------------------------------------------')
print ('The test data has {} rows and {} columns'.format(test.shape[0],test.shape[1]))

The train data has 5848 rows and 73 columns
---------------------------------------------
The test data has 1950 rows and 73 columns


In [136]:
print("Training set missing values:\n", train.isna().sum())
print("\nTest set missing values:\n", test.isna().sum())

Training set missing values:
 index                   0
budget_processed        0
popularity              0
release_date            0
runtime                 5
                       ..
original_language_uk    0
original_language_ur    0
original_language_vi    0
original_language_xx    0
original_language_zh    0
Length: 73, dtype: int64

Test set missing values:
 index                   0
budget_processed        0
popularity              0
release_date            0
runtime                 1
                       ..
original_language_uk    0
original_language_ur    0
original_language_vi    0
original_language_xx    0
original_language_zh    0
Length: 73, dtype: int64


In [137]:
train.dtypes

index                   float64
budget_processed        float64
popularity              float64
release_date            float64
runtime                 float64
                         ...   
original_language_uk      uint8
original_language_ur      uint8
original_language_vi      uint8
original_language_xx      uint8
original_language_zh      uint8
Length: 73, dtype: object

In [139]:
scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Modeling

In [140]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [142]:
# add missing dummy variables to testing set (in training only)
for column in X_train.columns:
    if column not in X_test.columns:
        X_test[column] = 0

In [150]:
X_train.isnull().values.any()

True

In [149]:
X_train

,index,budget_processed,popularity,release_date,runtime,genre_Comedy,genre_Drama,genre_Family,genre_Romance,genre_Thriller,genre_Action,genre_Animation,genre_Adventure,genre_Horror,genre_Documentary,genre_Music,genre_Crime,genre_Science_Fiction,genre_Mystery,genre_Foreign,genre_Fantasy,genre_War,genre_Western,genre_History,genre_TV_Movie,original_language_af,original_language_ar,original_language_bm,original_language_bn,original_language_ca,original_language_cn,original_language_cs,original_language_da,original_language_de,original_language_el,original_language_en,original_language_es,original_language_fa,original_language_fi,original_language_fr,original_language_he,original_language_hi,original_language_hu,original_language_id,original_language_is,original_language_it,original_language_ja,original_language_ka,original_language_kn,original_language_ko,original_language_lt,original_language_ml,original_language_mr,original_language_nb,original_language_nl,original_language_no,original_language_pl,original_language_pt,original_language_ro,original_language_ru,original_language_sr,original_language_sv,original_language_ta,original_language_te,original_language_th,original_language_tl,original_language_tr,original_language_uk,original_language_ur,original_language_vi,original_language_xx,original_language_zh
2926,2926.0,18000000.0,5.325766,1.188346e+18,90.0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5502,5502.0,28000000.0,2.304986,7.975584e+17,74.0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2809,2809.0,35000000.0,8.910462,7.851168e+17,89.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1025,1025.0,18000000.0,5.819403,1.383696e+18,91.0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
101,101.0,60000000.0,11.936460,1.008029e+18,135.0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5089,5089.0,12000000.0,4.757269,1.252886e+18,111.0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1247,1247.0,1500000.0,8.504511,1.176163e+18,78.0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5856,5856.0,150000000.0,29.158489,1.303344e+18,115.0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4681,4681.0,18000000.0,2.660367,2.450304e+17,98.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [147]:
y_train

2926     41098065.0
5502     18000000.0
2809      9771658.0
1025         4187.0
101     313542341.0
           ...     
5089     18000000.0
1247     30448000.0
5856     18000000.0
4681     18000000.0
3526     18000000.0
Name: revenue, Length: 5848, dtype: float64

In [143]:
reg = LogisticRegression().fit(X_train, y_train)
reg.score(X_test, y_test)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').